In [ ]:
# Célula 1 - Importação de Bibliotecas e Leitura do Arquivo CSV
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from d2l import tensorflow as d2l

# Lendo o arquivo CSV
file_path = 'winequalityN.csv'
data = pd.read_csv(file_path)

# Visualizando as primeiras linhas dos dados
data.head()


In [ ]:
# Célula 2 - Pré-processamento dos Dados
# Criando uma variável dummy para a coluna "type"
data = pd.get_dummies(data, columns=['type'], drop_first=True)

# Dividindo os dados em conjuntos de treinamento e validação (80% treinamento, 20% validação)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Verificando as dimensões dos conjuntos de treinamento e validação
train_data.shape, val_data.shape


In [ ]:
# Célula 3 - Preparação dos Dados para Treinamento
class LinearRegression(d2l.Module):
    """O modelo de regressão linear implementado com APIs de alto nível."""

    def __init__(self, lr):
        super().__init__()
        self.save_hyperparameters()
        initializer = tf.initializers.RandomNormal(stddev=0.01)
        self.net = tf.keras.layers.Dense(1, kernel_initializer=initializer)

class LinearRegressionData(d2l.DataModule):
    """Dados para regressão linear."""

    def __init__(self, nInputs, data, num_train=5197, num_val=1300, batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        self.X = data[:, 0:nInputs]
        self.y = data[:, nInputs]

@d2l.add_to_class(d2l.DataModule)
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    shuffle_buffer = tensors[0].shape[0] if train else 1
    return tf.data.Dataset.from_tensor_slices(tensors).shuffle(
        buffer_size=shuffle_buffer).batch(self.batch_size)

@d2l.add_to_class(LinearRegressionData)
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)


In [ ]:
# Célula 4 - Cálculo dos Pesos do Modelo
Xb = tf.concat((train_data.iloc[:, 1:].values, tf.ones((train_data.shape[0], 1), dtype=tf.float64)), axis=1)
y = train_data['quality'].values

W = tf.linalg.matvec(tf.linalg.matmul(tf.linalg.inv(tf.matmul(tf.transpose(Xb), Xb)), tf.transpose(Xb)), y)


In [ ]:
# Célula 5 - Métodos para Treinamento
@d2l.add_to_class(LinearRegression)
def forward(self, X):
    return self.net(X)

@d2l.add_to_class(LinearRegression)
def loss(self, y_hat, y):
    fn = tf.keras.losses.MeanSquaredError()
    return fn(y, y_hat)

@d2l.add_to_class(LinearRegression)
def configure_optimizers(self):
    return tf.keras.optimizers.SGD(self.lr)


In [ ]:
# Célula 6 - Treinamento
# Configuração do treinamento
lr = 1e-5  # Taxa de aprendizado

# Restante do código para o treinamento do modelo (não fornecido no texto original)
